In [1]:
from pydantic import BaseModel
from typing import List
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.element import Element

class Item(BaseModel):
    adminMetadata: AdminMetadata
    cdd: str
    cutter: str
    year: str
    shelf: str
    barcode: str

class Items_Schema(BaseModel):
  itemOf: Element
  items: List[Item]

In [21]:
obj = {
  "itemOf": {
    "label": "Physiological plant ecology",
    "lang": "pt-BR",
    "uri": "https://bibliokeia.com/resources/instance/bkc-2",
    "type": [
      "Instance"
    ]
  },
  "items": [
    {
      "adminMetadata": {
        "generationProcess": "BiblioKeia v.1",
        "status": {
          "label": "novo",
          "value": "mstatus:new"
        }
      },
      "cdd": "546.5",
      "cutter": "O48a",
      "year": "2023",
      "shelf": "E1-P1",
      "barcode": "23-0002"
    }
  ]
}

request = Items_Schema(**obj)
request.items

[Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 7, 18), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 7, 18, 15, 52, 19, 138033), status=Status(value='mstatus:new', label='novo')), cdd='546.5', cutter='O48a', year='2023', shelf='E1-P1', barcode='23-0002')]

In [27]:
request.items[0].adminMetadata.generationDate

datetime.datetime(2023, 7, 18, 15, 52, 19, 138033)

In [3]:
from pyfuseki import FusekiQuery

from api.src.schemas.settings import Settings

settings = Settings()

def GenerateId():

    collection = FusekiQuery(f'{settings.url}:3030', 'collection')

    sparql = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            SELECT ?g
            {graph ?g {?s bf:generationDate ?o}} 
            ORDER BY DESC(?o)
            LIMIT 1"""
    
    response = collection.run_sparql(sparql)
    response = response.convert()
    bindings = response['results']['bindings']

    if len(bindings) == 0:
        return { 'id': "bkc-1" }
    else:
        value = bindings[0]['g']['value']
        bk = int(value.split("-")[1]) + 1
        return { 'id': "bkc-"+str(bk) } 

In [20]:
response = GenerateId()
id = response['id']
id

'bkc-4'

In [10]:
from api.src.function.bibframe.Item.graphItem import MakeGraphItems
from pyfuseki import FusekiUpdate

collection = FusekiUpdate(f'{settings.url}:3030', 'collection')


for item in request.items: 
    response = GenerateId()
    id = response['id']
    graph = MakeGraphItems(item, request.itemOf, id)
    responseItem = collection.run_sparql(graph)
    print(responseItem)

{"requestedFormat" : 'json',
"response (a file-like object, as return by the urllib2.urlopen library call)" : {
	"url" : "http://localhost:3030/collection/update",
	"code" : "200",
	"headers" : Date: Tue, 18 Jul 2023 19:57:20 GMT
Connection: close
Fuseki-Request-Id: 168
Content-Type: application/json;charset=utf-8
Vary: Accept-Encoding

}}


In [11]:
from api.src.function.bibframe.Instance.hasItem import HasItem


upInstance = HasItem(request.itemOf.uri, id)
upInstance

'PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>\n        INSERT DATA\n        { GRAPH <https://bibliokeia.com/resources/instance/bkc-2> \n        { <https://bibliokeia.com/resources/instance/bkc-2>  bf:hasItem  <https://bibliokeia.com/resources/item/bkc-3> } }'

In [13]:
responseInstance = collection.run_sparql(upInstance)
responseInstance

In [ ]:
from api.src.function.solr.docItem import DocItem


DocItem(item, request.itemOf, id)

In [53]:
collectionUpdate = FusekiUpdate('http://localhost:3030', 'collection')

In [54]:
item = request.items[0]
graph = MakeGraphItems(item, "https://bibliokeia.com/resources/instance/bkc-2")

responseDelete = collectionUpdate.run_sparql(graph)
responseDelete.convert()


{'statusCode': 200, 'message': 'Update succeeded'}

In [14]:
import pysolr 

#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/collection/', timeout=10)

def DocItem(item, itemOf, id):

    instanceID = itemOf.uri.split("/")[-1]

    doc = {
        "id": id,
        "type": "Item",
        "heldBy": "Biblioteca do INPA",
        "cdd": item.cdd,
        "cutter": item.cutter,
        "year": item.year,
        "shelf": item.shelf, 
        "barcode": item.barcode,
        "itemOf": {'id': f"{id}/itemOf/{instanceID}",
                   'mainTitle': itemOf.label,
                   'uri': itemOf.uri}
        }
    
    instance = {
    'id': instanceID,
    'hasItem': {  "add": { 'id': f"{instanceID}/hasItem/{id}",
                          'uri': f"https://bibliokeia.com/resources/item/{id}",
                          'barcode': item.barcode }
    } }

    responseSolr = solr.add([doc, instance], commit=True)
    return responseSolr

In [16]:
responseSolr = DocItem(item, request.itemOf, id)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":287}}\n'

In [ ]:
{
  "id": "P11!S21",
  "_root_": "P11!prod",
  "manuals": { "add": { "id": "P11!D99",
                        "name_s": "Why Red Staplers Are the Best",
                        "content_t": "Once upon a time, Mike Judge ...",
                      } }
} 

In [82]:
instance = {
    'id': 'bkc-2',
    'hasItem': {  "add": { 'id': f"bkc-2/hasItem/i",
                          'uri': "url",
                          'barcode': "barcode"    }
    }
}
solr.add([instance], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":106}}\n'

In [85]:
request.items[0]

Item(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-06-27', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 27, 16, 0, 52, 826753), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='23-8486')], status=Status(value='mstatus:new', label='novo')), call='582.56 O48p 2023', shelf='E11-P05', barcode='23-8486')